In [1]:
from collections import defaultdict
from mysql.connector.errors import Error

import datetime
import sys
import json
import mysql.connector
import os
import glob
import re

In [2]:

fileslist=[]
dirlist= ['/media/data/sensemaking/dataset/social_graph/*.dat']
for dirl in dirlist:
    fileslist.extend(glob.glob(dirl,recursive=True))
    print(re.search('dataset/(.+?)/',dirl).group(1))
    print('directory files are adding to filelist: %s:'%dirl)
print('no.of file in directory: %d'%len(fileslist))
for file in fileslist:
    print(file)

social_graph
directory files are adding to filelist: /media/data/sensemaking/dataset/social_graph/*.dat:
no.of file in directory: 2
/media/data/sensemaking/dataset/social_graph/SE_collected_graph.dat
/media/data/sensemaking/dataset/social_graph/SE_collected_graph_p1.dat


In [7]:
d=defaultdict(list)
d1=defaultdict(list)
friend=[]
follow=[]
followers=[]
for file in fileslist:
    #print(file)
    with open(file) as f:
        content = f.readlines()
         # you may also want to remove whitespace characters like `\n` at the end of each line
    #content = [x.strip() for x in content]
    #print(len(content))
       
    for x in content:
        tokens = re.findall('\s+', x)
        k= len(tokens);
        if k==2:
        #print(name[i])
            str1= (x).split()
        
            if len(str1) == 3:
                friend.append((str1[0],str1[1]))
                follow.append((str1[0],str1[2]))
            elif len(str1) == 2:
                friend.append((str1[0],str1[1]))
                follow.append((str1[0],""))
        elif k==1:
            #print(name[i])
            str2= (x).split()
            if len(str2) == 2:
                friend.append((str2[0],""))
                follow.append((str2[0],str2[1]))
            elif len(str2) == 1:
                friend.append((str2[0],""))
                follow.append((str2[0],""))
                
#fnds::::::::::::::

for (k,v) in friend:
    d[k].append(v)
    
friends=[]   
friendlist=list(d.items())
friends_list=[]
friends_list_nodup=[]
for (k,v) in friendlist:
    # to eliminate space or emoty string in list [x for x in v if x] returnded
    #list join with the #
    id_list='#'.join( [x for x in v if x])
    for i in id_list.split("#"):
        friends.append((k,i))

firnd_set=set(friends)
print(len(friends))

print(len(firnd_set))

for i in firnd_set:
    friends_list.append(i)
print(len(friends_list))
for i in friends_list:
    j1= int(i[0])
    j2=int(int(0 if i[1] is '' else i[1]))
    friends_list_nodup.append((j1,j2))
print(len(friends_list_nodup))


# followers:::::::::  

for (k,v) in follow:
    d1[k].append(v)
followers=[]   
followerslist=list(d1.items())
followers_list=[]
followers_list_nodup=[]
for (k,v) in followerslist:
    id1_list='#'.join( [x for x in v if x])
    for i in id1_list.split("#"):
        followers.append((k,i))

followers_set=set(followers)
print(len(followers))

print(len(followers_set))
for i in followers_set:   
    followers_list.append(i)
for i in followers_list:
    j1= int(i[0])
    j2=int(int(0 if i[1] is '' else i[1]))
    followers_list_nodup.append((j1,j2))

print(len(followers_list))    
print(len(followers_list_nodup))

s1= friendsinsintosql(friends_list_nodup)        
print('frinds data insertion operation : %s'%s1)  
"""
s2= followersinsintosql(followers_list_nodup)
print('followers insertion operation : %s'%s2)
"""

16001262
8660970
8660970
8660970
11113
11113
11113
11113
friendsinsintosql method is started
8660970
(127042116, 336676648)
(1466320177, 241520774)
(1056130562, 2944535618)
(751580254330875904, 4762513523)
(2354268503, 619326360)
frinds data insertion operation : success


"\ns2= followersinsintosql(followers_list_nodup)\nprint('followers insertion operation : %s'%s2)\n"

In [6]:
def friendsinsintosql(friends_list_nodup):
    
    print("friendsinsintosql method is started")
    print(len(friends_list_nodup))
    for i in range(0,5):
        print(friends_list_nodup[i])
        
        
    cnx = mysql.connector.connect(user='root', password= 'root', host = 'localhost',database = 'twitterSchema')
    
    add_followee= "INSERT INTO followee(id_str, followee_id) VALUES (?, ?)" 
    
    try:
        cursor = cnx.cursor(prepared=True)                        
        cursor.executemany(add_followee,friends_list_nodup)
        cnx.commit()
        cursor.close()
        cnx.close()
    except Error as err:
        print("Something went wrong: {}".format(err))
        raise mysql.connector.Error(err)
        
    return 'success';

In [3]:
def followersinsintosql(followers_list_nodup):
    
    print("followersinsintosql method is started")
    print(len(followers_list_nodup))
       
        
    cnx = mysql.connector.connect(user='root', password= 'root', host = 'localhost',database = 'twitterSchema')
    
    add_followers= "INSERT INTO followers(id_str, followersid) VALUES (?, ?)" 
    
    try:
        cursor = cnx.cursor(prepared=True)                        
        cursor.executemany(add_followers,followers_list_nodup)
        cnx.commit()
        cursor.close()
        cnx.close()
    except Error as err:
        print("Something went wrong: {}".format(err))
        raise mysql.connector.Error(err)
        
    return 'success';